# Initializes the algorithm and shows the ideal classification

In [694]:
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg', 'pdf')
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split


import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.datasets import make_moons

# n = 500
# n_labeled = 10
alpha = 0.3
sigma = 0.2



/var/folders/0c/_n0_6h65185cts2w36thpb8m0000gn/T/ipykernel_2741/4112954314.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf')


In [695]:
# file = open('Y_input', 'wb')
# pickle.dump(Y_input, file)
# file.close()

In [696]:
# file = open('X', 'wb')
# pickle.dump(X, file)
# file.close()

In [697]:
# file = open('Y', 'wb')
# pickle.dump(Y, file)
# file.close()

In [698]:
file = open('Y_input', 'rb')
Y_input = pickle.load(file)

In [699]:
file = open('adj', 'rb')
adj = pickle.load(file)

In [700]:
file = open('rn_weight', 'rb')
rn_weight = pickle.load(file)

In [701]:
file = open('y_train', 'rb')
y_train = pickle.load(file)
n_labeled = len(y_train)

In [702]:
file = open('yy', 'rb')
yy = pickle.load(file)


In [703]:
file = open('X', 'rb')
X = pickle.load(file)


In [704]:
rn_weight = rn_weight[:n_labeled]
rn_weight = np.array(rn_weight)

In [705]:
adj = np.array(adj)
adj

array([[ 0.        , 15.30696299, 18.31393229, ..., 36.66343573,
        34.67631964, 35.66859996],
       [15.30696299,  0.        , 13.38640871, ..., 33.22316123,
        32.39044033, 32.58090141],
       [18.31393229, 13.38640871,  0.        , ..., 24.74783223,
        23.70176022, 23.37192389],
       ...,
       [36.66343573, 33.22316123, 24.74783223, ...,  0.        ,
        14.94038242, 10.6462108 ],
       [34.67631964, 32.39044033, 23.70176022, ..., 14.94038242,
         0.        , 12.3489074 ],
       [35.66859996, 32.58090141, 23.37192389, ..., 10.6462108 ,
        12.3489074 ,  0.        ]])

## Step 1: Construct the affinity matrix W

In [706]:
dm = adj
rbf = lambda x, sigma: math.exp((-x)/(2*(math.pow(sigma,2))))
vfunc = np.vectorize(rbf)
W = vfunc(dm, sigma)
np.fill_diagonal(W, 0)

## Step 2: Calculate S such as S = D^(-1/2)*W*D^(-1/2)

In [707]:
def calculate_S(W):
    d = np.sum(W, axis=1)
    D = np.sqrt(d*d[:, np.newaxis])
    return np.divide(W,D,where=D!=0)

S = calculate_S(W)

## Result of one iteration

In [708]:
n_iter = 10
F = np.dot(S, Y_input)*alpha + (1-alpha)*Y_input
for t in range(n_iter):
    F = np.dot(S, F)*alpha + (1-alpha)*Y_input
    F[:n_labeled,0] =  F[:n_labeled,0] *rn_weight
    F[:n_labeled,1] =  F[:n_labeled,1] *rn_weight
Y_result = np.zeros_like(F)
Y_result[np.arange(len(F)), F.argmax(1)] = 1

Y_v = np.argmax(Y_result, axis=1)
print(classification_report(yy, Y_v))



              precision    recall  f1-score   support

           0       0.95      0.98      0.96       125
           1       1.00      0.99      0.99       547

    accuracy                           0.99       672
   macro avg       0.97      0.99      0.98       672
weighted avg       0.99      0.99      0.99       672

